In [ ]:
# Training parameters

window = 672       # 1 week
stride = 4         # 1 hour
latent_dim = 10    # Autoencoder latent dimension
epochs = 150       # Number of epochs
batch_size = 8     # Batch size
M = 200            # Montecarlo

In [ ]:
# Reparametrization trick
def sample(args):
    z_mean, z_log_var = args
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1]), seed=seed)
    return z_mean + tf.exp(alpha * z_log_var) * eps

In [ ]:
# Building the model

from keras import backend as K
from tensorflow.keras import Input

input_shape = X_train.shape[1:]
output_shape = X_train.shape[1:]

###########
# ENCODER #
###########

encoder_input = tf.keras.Input(shape=input_shape)

x = tfkl.LSTM(64)(encoder_input)

x = tfkl.LSTM(latent_dim, return_sequences=False)(x)

# Latent representation: mean + log of std.dev.
z_mu = tfkl.Dense(latent_dim, name='latent_mu')(x) # Mean
z_sigma = tfkl.Dense(latent_dim, name='latent_sigma')(x) # Std.Dev. 

# Sampling a vector from the latent distribution
z = tfkl.Lambda(sample, output_shape=(latent_dim, ), name='z')([z_mu, z_sigma])

encoder = tfk.Model(encoder_input, [z_mu, z_sigma, z], name='encoder')
print(encoder.summary())

In [ ]:
###########
# DECODER #
###########

decoder_input = Input(shape=(latent_dim, ), name='decoder_input')
convlstm = tfkl.RepeatVector(window)(decoder_input)
convlstm = tfkl.LSTM(64, return_sequences=True)(convlstm)
decoder_output = tfkl.TimeDistributed(tfkl.Dense(output_shape[1]))(convlstm)

# Define and summarize decoder model
decoder = tfk.Model(decoder_input, decoder_output, name='decoder')
decoder.summary()

In [ ]:
# Sampling + Reparametrization trick
def sample_z1(z_m, z_l_v):
    z_mean = z_m 
    z_log_var = z_l_v
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1]), seed=seed)
    return z_mean + tf.exp(alpha * z_log_var) * eps

In [ ]:
class VAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tfk.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def forward(self, x):
        outputs = {}
        
        z_mu, z_log_var, _ = self.encoder(x)
        z = sample_z1(z_mu,z_log_var)
        reconstruction = self.decoder(z)
        
        outputs["z_mu"] = z_mu
        outputs["z_log_var"] = z_log_var
        outputs["z"] = z
        outputs["reconstruction"] = reconstruction
        
        return outputs

    def train_step(self, y_true):
        with tf.GradientTape() as tape:
            
            z_mu, z_log_var, _ = self.encoder(y_true)
            z = sample_z1(z_mu,z_log_var)
            y_predict = self.decoder(z)
          
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(y_true, y_predict), axis=1))
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mu) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result()
        }
    
    def test_step(self, data): #https://github.com/keras-team/keras-io/issues/38

        z_mu, z_log_var, _ = self.encoder(data)
        z = sample_z1(z_mu,z_log_var)
        y_predict = self.decoder(z)
          
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y_predict), axis=1))
            
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mu) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
        total_loss = reconstruction_loss + kl_loss
        
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss
        }

In [ ]:
vae = VAE(encoder, decoder)

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

vae.compile(optimizer=tfk.optimizers.Adam())
vae.fit(x = X_train,
        validation_data = (X_val, None),
        epochs=epochs, 
        batch_size=batch_size)
vae.fit(x = X_train,
        validation_data = (X_val, None),
        epochs=epochs, 
        batch_size=batch_size,
        callbacks=[tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True), tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-5)]
       )